# Custom Financial BERT Training Notebook

In [1]:
!pip install transformers datasets scikit-learn pandas nltk tqdm evaluate

In [3]:
!pip install "transformers==4.36.2" --upgrade --quiet


In [5]:
!pip install datasets evaluate --quiet


In [7]:
!pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension


SyntaxError: invalid syntax (1310192510.py, line 2)

In [9]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["HF_HUB_DISABLE_TF_WARNING"] = "1"

import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments
)
import evaluate
from datasets import Dataset, DatasetDict


/opt/anaconda3/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


## Upload Kaggle CSV

In [12]:
import pandas as pd

# Enter the file name manually after placing it in the notebook folder
df_kaggle = pd.read_csv("stock_data.csv")
df_kaggle.head()


,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1


In [14]:
label_map = {-1:0,0:1,1:2}
df_kaggle['label']=df_kaggle['Sentiment'].map(label_map)
df_kaggle=df_kaggle[['Text','label']]
df_kaggle.head()

,Text,label
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,2
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,2
2,user I'd be afraid to short AMZN - they are lo...,2
3,MNTA Over 12.00,2
4,OI Over 21.37,2


## Load PhraseBank

In [17]:
fp = "Sentences_50Agree.txt"

texts = []
labels = []

with open(fp, 'r', encoding='ISO-8859-1', errors='replace') as f:
    for line in f:
        line = line.strip()
        if '@positive' in line:
            texts.append(line.replace('@positive','').strip())
            labels.append(2)
        elif '@neutral' in line:
            texts.append(line.replace('@neutral','').strip())
            labels.append(1)
        elif '@negative' in line:
            texts.append(line.replace('@negative','').strip())
            labels.append(0)

df_phrase = pd.DataFrame({'Text': texts, 'label': labels})
df_phrase.head()


,Text,label
0,"According to Gran , the company has no plans t...",1
1,Technopolis plans to develop in stages an area...,1
2,The international electronic industry company ...,0
3,With the new production plant the company woul...,2
4,According to the company 's updated strategy f...,2


In [19]:
!pip install chardet

import chardet

with open(fp, 'rb') as f:
    raw = f.read()

chardet.detect(raw)


{'encoding': 'ISO-8859-1', 'confidence': 0.7299498732935502, 'language': ''}

In [21]:
df = pd.concat([df_phrase, df_kaggle], ignore_index=True)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)
dataset = DatasetDict({'train':train_ds,'test':test_ds})

In [23]:
SEQ_LEN=256
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
def tok(batch): return tokenizer(batch['Text'], truncation=True, padding='max_length', max_length=SEQ_LEN)
enc = dataset.map(tok, batched=True).remove_columns(['Text']).rename_column('label','labels')
enc.set_format('torch')

/opt/anaconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/8509 [00:00<?, ? examples/s]

Map:   0%|          | 0/2128 [00:00<?, ? examples/s]

In [25]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

/opt/anaconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
!pip install accelerate -U


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
training_args = TrainingArguments(
    output_dir='./custom_finbert', evaluation_strategy='epoch', save_strategy='epoch',
    learning_rate=3e-5, per_device_train_batch_size=16, per_device_eval_batch_size=16,
    num_train_epochs=3, weight_decay=0.01, logging_steps=50)
acc = evaluate.load('accuracy'); f1 = evaluate.load('f1')
def metrics(p):
    logits, labels = p
    preds = logits.argmax(-1)
    return {'accuracy':acc.compute(predictions=preds, references=labels)['accuracy'],
            'f1_weighted':f1.compute(predictions=preds, references=labels, average='weighted')['f1']}
trainer = Trainer(model=model, args=training_args,
                  train_dataset=enc['train'], eval_dataset=enc['test'], compute_metrics=metrics)
trainer.train()

TypeError: Accelerator.__init__() got an unexpected keyword argument 'dispatch_batches'

In [ ]:
trainer.evaluate()